# PubChem

In this notebook we took structures from PDBbind core-set to find UniProt IDs for their corresponding protein. Next we mapped UniProt IDs to Gene IDs using [UniProt ID mapping](http://www.uniprot.org/uploadlists/) and found all matching PubChem Assay IDs. After downloading data from [PubChem](https://pubchem.ncbi.nlm.nih.gov/), we cleaned it and compared to data from [ChEMBL](https://www.ebi.ac.uk/chembl/). Finally we saved the data to csv. 

In [1]:
import os
import re
import requests
import pandas as pd
from lxml import etree
from tqdm import tqdm_notebook
from contextlib import redirect_stderr

import PubChem as pubchem

## Loading pdbbind core-set information

#### Cluster ID

In [2]:
f = './refined-set/index/INDEX_core_cluster.2016'
clusters = pd.read_csv(f, sep='\s+', usecols=[0, 5], 
                       names=['pdb_id', 'cluster_id'], comment='#')
clusters.head()

,pdb_id,cluster_id
0,1ps3,3
1,3dx1,3
2,3d4z,3
3,3dx2,3
4,3ejr,3


#### UniProt ID

In [3]:
f = './refined-set/index/INDEX_general_PL_name.2016'
uniprot = pd.read_csv(f, sep='\s+', usecols=[0, 2], 
                      names=['pdb_id', 'uniprot_id'], comment='#')
uniprot.head()

,pdb_id,uniprot_id
0,3eql,Q9Z9H6
1,1zyr,Q5SHR6
2,3dxj,Q5SHR6
3,4zh4,P0A7Z4
4,4zh3,P0A7Z4


#### Merge

In [4]:
data_pdb = clusters.merge(uniprot, on='pdb_id')
data_pdb.head()

,pdb_id,cluster_id,uniprot_id
0,1ps3,3,Q24451
1,3dx1,3,Q24451
2,3d4z,3,Q24451
3,3dx2,3,Q24451
4,3ejr,3,Q24451


## ID mapping

#### From UniProt ID to Gene ID

In [5]:
# all uniprot IDs 
uniprot_ids = set(data_pdb['uniprot_id'].tolist())

In [6]:
text_file = open('UniProt_IDs.txt', 'w')
tmp = re.sub(r'[,\'{}]', '', str(uniprot_ids))
text_file.write(tmp)
text_file.close()

For mapping I used http://www.uniprot.org/uploadlists/.

In [7]:
mapping_gid = pd.read_csv('./GeneName.txt', sep='\t')
mapping_gid.head()

,From,To
0,P00519,25
1,P15207,24208
2,P19491,29627
3,P11309,5292
4,Q9Y233,10846


In [8]:
gene_ids = set(mapping_gid['To'])

## Download data
#### For every Gene ID find matching Assay ID(s) and then for every Assay ID download csv from PubChem

In [9]:
# path to store information
directory = './aid_files/'
os.system('mkdir -p %s' % directory)

0

In [10]:
for gene_id in gene_ids:

    aids = pubchem.get_AIDs(gene_id)
    
    if aids[0] != 'Status:':
        for aid in aids:
            pubchem.download_aid_csv(aid, directory)
        

In [11]:
aid_list = []
gen_list = []

for gene_id in gene_ids:

    aids = pubchem.get_AIDs(gene_id)
    if aids[0] != 'Status:':
        aid_list += aids
        gen_list += [gene_id]*len(aids)
    
gen_aid = pd.DataFrame({'aid': aid_list, 'gen': gen_list})
gen_aid.head()

,aid,gen
0,1433,3716
1,1982,3716
2,256646,3716
3,277462,3716
4,339778,3716


### Example

In [12]:
pd.read_csv('./aid_files/1141064.csv', index_col=0)

,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,IC50,SEI,BEI,LE,LLE,IC50 activity comment,IC50 standard flag,IC50 qualifier,IC50 published value,IC50 standard value,IC50 data validity
PUBCHEM_RESULT_TAG,,,,,,,,,,,,,,,,,
RESULT_TYPE,NaN,NaN,NaN,NaN,NaN,NaN,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,STRING,INTEGER,STRING,FLOAT,FLOAT,STRING
RESULT_DESCR,NaN,NaN,NaN,NaN,NaN,NaN,IC50 PubChem standard value,Surface Efficiency Index(nM),Binding Efficiency Index(nM),Ligand Efficiency,Lipophilic Ligand Efficiency,IC50 activity comment,IC50 standard flag,IC50 qualifier,IC50 published value,IC50 standard value,IC50 data validity
RESULT_UNIT,NaN,NaN,NaN,NaN,NaN,NaN,MICROMOLAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MICROMOLAR,NANOMOLAR,NaN
RESULT_IS_ACTIVE_CONCENTRATION,NaN,NaN,NaN,NaN,NaN,NaN,TRUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,194146212.0,56684144.0,Active,NaN,NaN,NaN,4.1,4.43,13.13,0.25,5.23,NaN,NaN,=,4.1,4100,NaN
2,242634058.0,57381960.0,Unspecified,NaN,NaN,NaN,117,NaN,NaN,NaN,NaN,NaN,NaN,=,117,117000,Outside typical range


## Find csv with good bioactivity types

In [13]:
good_aids = []
activity = []

# redirect stderr to file
with open('stderr.log', 'w') as stderr, redirect_stderr(stderr):
    for gene_id in gene_ids:

        aids = pubchem.get_AIDs(gene_id)

        for aid in aids:

            try:
                data = pd.read_csv('./aid_files/%s.csv' % aid, index_col=0)
            except:
                continue

            for col in data:
                if re.search(r'(IC50)', col):
                #if re.search(r'[(EC50)(IC50)(Inhibition)]', col):  # (Activity)(Ka)(Kd)(Ki)(Km)
                    activity.append(col)
                    good_aids.append(aid)
                    #break
                    

In [14]:
good_aids = list(set(good_aids))
len(good_aids)

2771

In [15]:
set(activity)

{'Activity At IC50',
 'Ba/F3 Cytotoxicity (IC50) ',
 'High Concentration (5 x IC50)',
 'I/IC50 activity comment',
 'I/IC50 published value',
 'I/IC50 qualifier',
 'I/IC50 standard flag',
 'I/IC50 standard value',
 'IC50',
 'IC50 Binding Efficiency Index(nM)',
 'IC50 Binding Efficiency Index(nM).1',
 'IC50 Hill slope',
 'IC50 Ligand Efficiency',
 'IC50 Ligand Efficiency.1',
 'IC50 Lipophilic Ligand Efficiency',
 'IC50 Lipophilic Ligand Efficiency.1',
 'IC50 R-squared',
 'IC50 Surface Efficiency Index(nM)',
 'IC50 Surface Efficiency Index(nM).1',
 'IC50 activity comment',
 'IC50 activity comment.1',
 'IC50 binding domains',
 'IC50 binding domains.1',
 'IC50 blank mean',
 'IC50 blank percent CV',
 'IC50 blank standard deviation',
 'IC50 control mean',
 'IC50 control percent CV',
 'IC50 control standard deviation',
 'IC50 data validity',
 'IC50 data validity.1',
 'IC50 max concentration',
 'IC50 min concentration',
 'IC50 number of blank wells',
 'IC50 number of control wells',
 'IC50 perc

In [16]:
data = pd.read_csv('./aid_files/%s.csv' % good_aids[0], index_col=0)
data

,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,IC50,IC50 activity comment,IC50 standard flag,IC50 qualifier,IC50 published value,IC50 standard value
PUBCHEM_RESULT_TAG,,,,,,,,,,,,
RESULT_TYPE,NaN,NaN,NaN,NaN,NaN,NaN,FLOAT,STRING,INTEGER,STRING,FLOAT,FLOAT
RESULT_DESCR,NaN,NaN,NaN,NaN,NaN,NaN,IC50 PubChem standard value,IC50 activity comment,IC50 standard flag,IC50 qualifier,IC50 published value,IC50 standard value
RESULT_UNIT,NaN,NaN,NaN,NaN,NaN,NaN,MICROMOLAR,NaN,NaN,NaN,MICROMOLAR,NANOMOLAR
RESULT_IS_ACTIVE_CONCENTRATION,NaN,NaN,NaN,NaN,NaN,NaN,TRUE,NaN,NaN,NaN,NaN,NaN
1,242637107.0,53389651.0,Unspecified,NaN,NaN,NaN,10,NaN,NaN,>,10,10000


In [17]:
data[[x for x in data.columns if x in activity]]

,IC50,IC50 activity comment,IC50 standard flag,IC50 qualifier,IC50 published value,IC50 standard value
PUBCHEM_RESULT_TAG,,,,,,
RESULT_TYPE,FLOAT,STRING,INTEGER,STRING,FLOAT,FLOAT
RESULT_DESCR,IC50 PubChem standard value,IC50 activity comment,IC50 standard flag,IC50 qualifier,IC50 published value,IC50 standard value
RESULT_UNIT,MICROMOLAR,NaN,NaN,NaN,MICROMOLAR,NANOMOLAR
RESULT_IS_ACTIVE_CONCENTRATION,TRUE,NaN,NaN,NaN,NaN,NaN
1,10,NaN,NaN,>,10,10000


In [18]:
for aid in good_aids:
        
        data = pd.read_csv('./aid_files/%s.csv' % aid, index_col=0)
                
        if 'PUBCHEM_SID' not in data.columns:
            print(aid, col)

In [19]:
data = pd.read_csv('./aid_files/%s.csv' % '721825', index_col=0)
data[[x for x in data.columns if x in activity]]

,IC50 activity comment,IC50 standard flag,IC50 qualifier,IC50 published value,IC50 standard value,IC50 data validity,IC50 binding domains
PUBCHEM_RESULT_TAG,,,,,,,
RESULT_TYPE,STRING,INTEGER,STRING,FLOAT,FLOAT,STRING,STRING
RESULT_DESCR,IC50 activity comment,IC50 standard flag,IC50 qualifier,IC50 published value,IC50 standard value,IC50 data validity,IC50 binding domains
RESULT_UNIT,NaN,NaN,NaN,ug ml-1,ug.mL-1,NaN,NaN
1,NaN,NaN,=,92.7,92.7,Outside typical range,NaN


In [20]:
# redirect stderr to file
with open('stderr.log', 'w') as stderr, redirect_stderr(stderr):
    for gene_id in list(gene_ids):

        aids = list(gen_aid.loc[gen_aid['gen'] == gene_id]['aid'])
        data = pubchem.gene_id_to_data_frame(gene_id, aids)
        data.to_csv('./pubchem/pubchem_%s.csv' % gene_id, index=False)
            

In [21]:
pd.read_csv('./pubchem/pubchem_100008972.csv')

,bioactivity,gene_id,pubchem_aid,pubchem_sid,value
0,IC50,100008972,260787,103178995.0,114.0000
1,IC50,100008972,266475,103178995.0,114.0000
2,IC50,100008972,270947,103485311.0,1.2100
3,IC50,100008972,270948,103485311.0,0.7600
4,IC50,100008972,404873,103165295.0,43.0000
5,IC50,100008972,408492,103641450.0,11700.0000
6,IC50,100008972,426720,103684498.0,10000.0000
7,IC50,100008972,427071,103178995.0,283.0000
8,IC50,100008972,427132,103625236.0,NaN
9,IC50,100008972,436293,103178995.0,83.1000
